# <font color='red'>Backtest vectorizado para una estrategia 'X' definida.</font>

### Hackeado by J3viton (learning BackTesting 2021).

---

En este notebook vamos a automatizar el backtesting para estrategias. Previamente tenemos que definir **la estrategia** usando el template (import estrategia_XX as rg), donde definimos la clase estrategia que contiene los metodos de analisis, entreda y salida de mercado. Este módulo se programa y adapta para cada estrategia. Luego en el metodo 'analisis' se crea un fichero excel con las metricas y rendimientos para evaluar la estrategia.

Vamos a usar el módulo Pandas, para realizar un backtest vectorizado, es decir calcular el resultado del backtest sin iterar sobre las filas (barras) de la serie temporal.

Posteriormente graficamos y mostramos los parametros generales que nos permiten analizar la estrategia.

#### J3Viton  2021

link a la base:

https://github.com/Python-para-Trading/Webinars-Docs/blob/master/Webinar%202/Webinar%202%20-%20Backtest%20con%20Pandas%20v%202.ipynb.

---

# DEFINICION DEL SISTEMA

Como es usual, importamos los módulos que vamos a necesitar para las distintas tareas, y preparamos Matplotlib para una mejor visualización de los gráficos en el notebook.

Se importa un modulo llamado analisis, en realidad se trata de un archivo .py con algunas funciones de apoyo para el trabajo, debe estar guardado en la misma carpeta que este notebook.
Se importa el módulo 'estrategia_XX', donde se define la estrategia de entreda y salida

Es importante tener clara la lógica del sistema, pues debemos plasmarla de forma sencilla y correcta en el dataframe para realizar el backtest del mismo.


In [1]:
import analisis
import estrategia_XX as rg  #cambiar segun el módulo con la estrategia implementado
import pandas as pd
import datetime as dt
from time import time
import yfinance as yf
import numpy as np

{'can_read_all_group_messages': False, 'id': 1473252352, 'is_bot': True, 'first_name': 'vital_bot', 'username': 'vital_quant_bot', 'supports_inline_queries': False, 'can_join_groups': True}


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 18, 8
plt.style.use('ggplot')

from IPython.display import display, Markdown

***1.- Rango fechas a analizar***

Definimos el rango global de datos historicos que vamos a evaluar.
Definimos la ventana que vamos a ir desplazando por todo el espectro para ir analilazando como se comporta la estrategia. La funion 'analisis' nos vale para tiempo real y para backtesting, para back le pasamos la ventana como si la fecha de fin de la ventana fuera la fecha de hoy.
Tener en cuenta que la 'ventana' tiene uqe tener una anchura que nos permita hacer los calculos en rolling (ejemplo EMA 200)

In [3]:
# Rango completo para backTesting
start2 =dt.datetime(2002,1,2)
end2   =dt.datetime(2021,11,18)
start_G= start2.strftime("%Y-%m-%d")
end_G  =   end2.strftime("%Y-%m-%d")
TOTAL_len= (end2-start2).days
print('Tamaño timeseries global a analizar:  ', TOTAL_len, 'sesiones')

#ventana de analisis 200 sesiones
startWindow2 =dt.datetime(2002,1,5)
endWindow2   =startWindow2 + dt.timedelta(days=1000) #ventana grande para que se puedan hacer los calculos de EMA200
startWindow= startWindow2.strftime("%Y-%m-%d")
endWindow  =   endWindow2.strftime("%Y-%m-%d")
window_len= (endWindow2-startWindow2).days
print('Tamaño de la ventana a analizar paso a paso:  ', window_len, 'sesiones')

Tamaño timeseries global a analizar:   7260 sesiones
Tamaño de la ventana a analizar paso a paso:   1000 sesiones


In [4]:
 #dff = pd.DataFrame(columns=('Close','Volume', 'Senal', 'Dif_Close', 'Retornos','Capital'))

***2.- Descargamos los datos para el marco Global***

In [5]:
instrumento = 'ACX.MC'    #Vamos título a título. Mejora: Conjunto de títulos

In [6]:
dff = yf.download(instrumento, start_G,end_G)

[*********************100%***********************]  1 of 1 completed


In [7]:
dff.dropna(inplace=True)  
dff.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,9.3500,9.8725,9.3375,9.7750,4.279621,747660
2002-01-03,9.8475,10.0000,9.7125,9.7125,4.252256,2695880
2002-01-04,9.7125,9.9975,9.6875,9.9975,4.400826,1297372
2002-01-07,9.9100,10.0625,9.8975,9.9750,4.390919,921060
2002-01-08,9.9175,9.9975,9.6800,9.9375,4.374414,1472784


***3.- Creamos la clase strategy que tiene toda la lógica***

In [8]:
regreMedia= rg.StrategyClass()    #Creamos la clase

***4.- Recorremos el dataframe con valores buscando las señales de la estrategia***

In [9]:
dff.index

DatetimeIndex(['2002-01-02', '2002-01-03', '2002-01-04', '2002-01-07',
               '2002-01-08', '2002-01-09', '2002-01-10', '2002-01-11',
               '2002-01-14', '2002-01-15',
               ...
               '2021-11-04', '2021-11-05', '2021-11-08', '2021-11-09',
               '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15',
               '2021-11-16', '2021-11-17'],
              dtype='datetime64[ns]', name='Date', length=5087, freq=None)

In [10]:
dff.loc[[endWindow],['Close']]


,Close
Date,
2004-10-01,11.42


## Proceso de backTesting ##

En este 'for' vamos recorriendo la muestra del historico de datos (TOTAL), desplazando una sesión hacia el futuro 
en cada iteracion. Vamos pasando la movilola del pasado sesión a sesión por el análisis descrito en Strategy class.

Ouput:
El sistema registra los siguientes parámetros:

.-
.-
.-


In [11]:
startWindow

'2002-01-05'

In [12]:
tiempo_inicial = time()   # Tomamos tiempos para ver cuanto tarda en hacer la estrategia

In [13]:
dfe = pd.DataFrame({'A' : []})   #df empty

***En este 'for' desplazamos la ventana sesión a sesión a lo largo de todo el rango de fechas. Dejamos que las funciones de la clase estrategia hagan el trabajo de comprar//vender y anotar***


In [14]:
#Para pruebas
#TOTAL_len =1000

for i in range(TOTAL_len):
    endWindow3   =endWindow2 + dt.timedelta(days=i) 
    endWindow    =endWindow3.strftime("%Y-%m-%d")
    print ('end date:', endWindow)
    
    if(endWindow in dff.index):
        df_aux= dff.loc[startWindow:endWindow]       #voy pasando los datos desplazando la ventana
        
        recogo = regreMedia.analisis(instrumento, startWindow, endWindow, df_aux) #Llamada a la clase estrategia. LA CLAVE DE TODO!!!
        
        print ('................................................Analizando, muestra', i, 'de', TOTAL_len, 'fecha', endWindow)
        
        """
        Tratamiento de los returns, mejor en las funciones de analisis que hacer aquí.
        if(recogo['senal'] == 1 ):
            dff.loc[[endWindow],['Senal']]= 1
        elif(recogo['senal'] == 0 ):
            dff.loc[[endWindow],['Senal']]= 0
        elif(recogo['senal'] == -1 ):
            dff.loc[[endWindow],['Senal']]= -1            
        elif(recogo['senal'] == 503 ):
            dff.loc[[endWindow],['Senal']]= 0          
        """
            
    else:
        print('..............Día sin sesión, next please')

        

end date: 2004-10-01
fichero no existe
 datos desde archivo
Coefficients: 
 [0.0016235]
Independent term: 
 9.045478192807805
Coefficients: 
 [0.01445821]
Independent term: 
 9.069295920471646
Coefficients: 
 [0.00909476]
Independent term: 
 8.8688621544385
  instrumento  long_short_out        date     precio
0      TEF.MC               0  2021-10-19   3.664000
1      IBE.MC               0  2015-05-05   5.841000
2     BBVA.MC               1  2021-09-20   5.248000
3      san.MC               0  2008-01-21   9.754196
4      ACX.MC               0  2005-06-30  11.270000
5      SAB.MC               0  2013-03-25   1.080511
................................................Analizando, muestra 0 de 7260 fecha 2004-10-01
end date: 2004-10-02
..............Día sin sesión, next please
end date: 2004-10-03
..............Día sin sesión, next please
end date: 2004-10-04
 datos desde archivo
Coefficients: 
 [0.00163927]
Independent term: 
 9.041730773022815
Coefficients: 
 [0.01443052]
Independent 

C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:825: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")
C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:825: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence thi

  instrumento  long_short_out        date     precio
0      TEF.MC               0  2021-10-19   3.664000
1      IBE.MC               0  2015-05-05   5.841000
2     BBVA.MC               1  2021-09-20   5.248000
3      san.MC               0  2008-01-21   9.754196
4      ACX.MC               0  2005-06-30  11.270000
5      SAB.MC               0  2013-03-25   1.080511
................................................Analizando, muestra 6 de 7260 fecha 2004-10-07
end date: 2004-10-08
 datos desde archivo
Coefficients: 
 [0.00170454]
Independent term: 
 9.026165452222333
Coefficients: 
 [0.01437858]
Independent term: 
 9.133768298305682
Coefficients: 
 [0.00935415]
Independent term: 
 8.880516649462217
  instrumento  long_short_out        date     precio
0      TEF.MC               0  2021-10-19   3.664000
1      IBE.MC               0  2015-05-05   5.841000
2     BBVA.MC               1  2021-09-20   5.248000
3      san.MC               0  2008-01-21   9.754196
4      ACX.MC             

C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:825: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")
C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:825: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence thi

 datos desde archivo
Coefficients: 
 [0.0017523]
Independent term: 
 9.014706897746498
Coefficients: 
 [0.01390122]
Independent term: 
 9.222678737355702
Coefficients: 
 [0.009587]
Independent term: 
 8.895342711085265
  instrumento  long_short_out        date     precio
0      TEF.MC               0  2021-10-19   3.664000
1      IBE.MC               0  2015-05-05   5.841000
2     BBVA.MC               1  2021-09-20   5.248000
3      san.MC               0  2008-01-21   9.754196
4      ACX.MC               0  2005-06-30  11.270000
5      SAB.MC               0  2013-03-25   1.080511
................................................Analizando, muestra 17 de 7260 fecha 2004-10-18
end date: 2004-10-19
 datos desde archivo
Coefficients: 
 [0.00176149]
Independent term: 
 9.012490909877547
Coefficients: 
 [0.01378189]
Independent term: 
 9.243326496437414
Coefficients: 
 [0.00962874]
Independent term: 
 8.898805236312649
  instrumento  long_short_out        date     precio
0      TEF.MC     

...

Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 2889 de 7260 fecha 2012-08-29
end date: 2012-08-30
 datos desde archivo
Coefficients: 
 [0.00086336]
Independent term: 
 11.236401915007852
Coefficients: 
 [-0.01301142]
Independent term: 
 10.670465883069962
Coefficients: 
 [-0.00840834]
Independent term: 
 11.048703933297185


C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:825: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")


Empty DataFrame
Columns: [A]
Index: []
................................................Analizando, muestra 2890 de 7260 fecha 2012-08-30
end date: 2012-08-31
 datos desde archivo
Coefficients: 
 [0.00085927]
Independent term: 
 11.240119069742173
Coefficients: 
 [-0.01318766]
Independent term: 
 10.685272144915453
Coefficients: 
 [-0.0084559]
Independent term: 
 11.04508769659687


C:\Users\INNOVACION\Documents\J3\100.- cursos\Quant_udemy\programas\Projects\template_B0\quant_j3_lib.py:825: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  sheet_name="data")


PermissionError: [Errno 13] Permission denied: 'logACX.MC.xls'

***Regogemos los datos de las entradas que ha realizado la Estrategia***

In [ ]:
data=rg.StrategyClass.dfLog
#data.dropna(inplace=True) 

In [ ]:
data.tail()

In [ ]:
data['Senal'].plot(title = 'Señales de compra de la estrategia ' )
# Ploteando una parte
#data.loc['2010':'2022','Senal'].plot(title = 'Señales Regresión a la media ' +instrumento,xlim=('2010','2022'))

In [ ]:
tiempo_final = time() 
 
tiempo_ejecucion = tiempo_final - tiempo_inicial
 
print ('El tiempo de ejecucion fue:',tiempo_ejecucion,"segundos") #En segundos

In [ ]:
data.loc['2011-01-04':'2019-11-04']

In [ ]:
#Guardamos el resultado del analisis en un pickle
#dff.to_pickle("./primerBack_IBE.pkl")

***3.- Ingeniería de datos para calcular la bondad de la estrategia***

In [ ]:

data['Dif_Close'] = data.Price.pct_change()
data['Retornos'] = data.Dif_Close * data.Senal.shift(1)   
data['Capital'] = (data.Retornos + 1).cumprod() * 100
    
rg.StrategyClass.dfLog=data  #Ojo a esta liena, no me cuadra.

#quant_j.salvarExcel(StrategyClass.dfLog, "log"+instrumento)   
#data.to_pickle('almacen')    #df = pd.read_pickle(file_name)

>Para calcular los retornos del sistema, calculamos la diferencia relativativa entre el precio de cierre y el del día anterior. Para posteriormente multiplicarlo por la señal que del día anterior que nos indicaba la posición a tomar (si estaba comprado, sumo beneficio).

In [ ]:
data.tail()

In [ ]:
data[60:]

>El capital, por comodidad, lo calcularemos en base 100, es decir como si iniciaramos la inversión con 100 unidades monetarias. Para su calculo arrastramos el producto acumulado de los retornos mas 1, multiplicados como dijimos por 100.

Pero para observar gráficamente la evolución del sistema necesitamos un gráfico con mas información. Para ello definimos una función que nos mostrará un gráfico con la evolución de nuestra estrategia, comparandola con la del activo. En un subgráfico se visualizará el drawdown del sistema, comparado de nuevo con el del activo. Y por útlimo las posiciones que toma el sistema.

In [ ]:
def grafico (df):
    estudio = df.copy()
    DD_bh, maxDD, maxDD_ini, maxDD_fin = analisis.DrawDown(estudio.Dif_Close[60:], info = False) 
    DD, maxDD, maxDD_ini, maxDD_fin = analisis.DrawDown(estudio.Retornos.fillna(0), info = False) 

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(18, 12), gridspec_kw = {'height_ratios':[3, 1, 1]})
    fig.suptitle('Estrategia vs B & H', fontsize=20)

    #Primer plot analiza la evolcuion de una inversion de 100€ en buy&hold y la estrategia)
    ax1.plot(estudio.Capital)
    ax1.plot((estudio.Price) * (100/estudio.Price[1]))
    ax1.set_title('Capital')
    ax1.legend(['Estrategia','Buy & Hold'])

    ax2.plot(DD*100, c='0.5')
    ax2.plot(DD_bh*100, c='y')
    ax2.fill_between(DD.index, 0, DD*100, color='0.7')
    ax2.set_title('Drawdown')
    ax2.legend(['Estrategia','Buy & Hold'])

    ax3.plot(estudio.Senal, c='orange')
    #ax3.fill_between(estudio.index, 0, estudio.Senal*100, color='orange')
    ax3.set_title('Posición')

    plt.show()
    return

In [ ]:
grafico(data)

---

In [ ]:
#data[55:70]

---

# ANALISIS DEL SISTEMA

retornos=data.Retornos
retornos
retorno_acumulado = retornos.fillna(0).add(1).cumprod()   
retorno_acumulado
dt_inicial = dt.datetime.strptime(retornos.index[1], '%Y-%m-%d')
dt_final = dt.datetime.strptime(retornos.index[-1], '%Y-%m-%d')
cagr = (retorno_acumulado[-1] / retorno_acumulado[1]) ** (365 / (dt_final - dt_inicial).days ) - 1
cagr

In [ ]:
Beneficio_Bruto = data.Capital[-1] - data.Capital[1]
print ('Beneficio Bruto : {:.2f} €  con una inverison de 100€'.format(Beneficio_Bruto))

In [ ]:
CAGR = analisis.CAGR(data.Retornos)
print ('CAGR : {:.2f}%'.format(CAGR * 100))

In [ ]:
data.Retornos=data.Retornos.replace(0,np.e)  #Quito los cero (no sepuede hacer logariotmo de cero), pero no se que poner
Sharpe = CAGR / (np.log(data.Retornos + 1).std(skipna=True) * 252 ** 0.5)
print ('Sharpe : {:.3f}'.format(CAGR ))

---


# PRUEBA FUERA DE LA MUESTRA

periodo_rapido = 5

periodo_lento = 140

out_sample= qd_data[data.shape[0]:].copy()

out_sample['Dif_Close'] = out_sample.Close.pct_change()
out_sample['Dif_Open'] = out_sample.Close.pct_change()  #era Cloe


out_estr, out_resultado = estrategia_medias (out_sample, periodo_rapido, periodo_lento)


grafico(out_estr)

---